<a href="https://colab.research.google.com/github/thofes/judging_SJ_BA/blob/main/Mark_13_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Connect GDrive
from google.colab import drive
drive.mount('/content.gdrive')

#init Repository - YOLOv5
%cd /content


!git clone https://github.com/thofes/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install


#init Repository - OpenPose
import os
from os.path import exists, join, basename, splitext

#Change the drive to my mounted gdrive
%cd /content

git_repo_url = 'https://github.com/thofes/openpose'
project_name = splitext(basename(git_repo_url))[0]

if 1 or not exists(project_name):
  !rm -rf openpose
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  print("install new CMake becaue of CUDA10")
  cmake_version = 'cmake-3.20.2-linux-x86_64.tar.gz'
  if not exists(cmake_version):
    !wget -q 'https://cmake.org/files/v3.20/{cmake_version}'
  !tar xfz {cmake_version} --strip-components=1 -C /usr/local

  print("clone openpose")
  !git clone -q --depth 1 $git_repo_url
  print("install system dependencies")
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  print("build openpose")
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

Mounted at /content.gdrive
/content
Cloning into 'yolov5'...
remote: Enumerating objects: 11171, done.
remote: Total 11171 (delta 0), reused 0 (delta 0), pack-reused 11171
Receiving objects: 100% (11171/11171), 11.14 MiB | 21.81 MiB/s, done.
Resolving deltas: 100% (7717/7717), done.
/content/yolov5
     |████████████████████████████████| 596 kB 5.5 MB/s 
/content
install new CMake becaue of CUDA10
clone openpose
install system dependencies
Selecting previously unselected package libgflags2.2.
(Reading database ... 156210 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Select

In [ ]:
#try 02.04.2022 10:52 (mark12)

#Update
# 28 < x < 75 
# abs < 30
# Lückenhafte Bilder werden besser

#sollte nun final sein - Test 02.04.2022 16:53 - erfolgreich

import os
from os.path import exists, join, basename, splitext
import math
from collections import OrderedDict
import json
import numpy as np
import cv2


#Aufbau Json File genau anschauen

json_path = "/content.gdrive/My Drive/Manually_Labeled_09042022/data_Manually_Labeled_09042022.json"

# dictionary to store mapping, labels, and Keypoints
dataset = {
    "labels":[],
    "keypoints":[]
}

PATH = "/content.gdrive/My Drive/Manually_Labeled_09042022"
label= []
try:
    label = next(os.walk(PATH))[1]
except StopIteration:
    pass
#data["labels"].append(label)
print(label)

for l in label:
  Path_jumpers = PATH + '/' + l
  print(Path_jumpers)
  Jumpers = []
  try:
    Jumpers = next(os.walk(Path_jumpers))[1]
  except StopIteration:
    pass
  print("Anzahl: " , len(Jumpers))

  for j in Jumpers:
    Path_Pic = Path_jumpers + '/' + j
    print(Path_Pic)

    %cd /content/yolov5
    !python detect.py --save-crop --source '{Path_Pic}' --project '{Path_Pic}' --save-txt


    with open(Path_Pic + '/exp/predictions.json', "r") as fp:
        data = json.load(fp)

    # convert lists to numpy arrays
    x_center = np.array(data["x_center"])
    y_center = np.array(data["y_center"])
    x1 = np.array(data["x1"])
    x2 = np.array(data["x2"])
    y1 = np.array(data["y1"])
    y2 = np.array(data["y2"])
    det = np.array(data["det"])
    width = 1920
    hight = 1080




    #detect movement mit check vor und zurück


    athlet_det_index = []


    for y in range(21):
      #print("loop" , y)
      if det[y] != 0: #wird eventuellenötigt, falls keine detections gemacht wurden
        #print("detvorhanden")
        distance_next = []
        distance_index_next = []
        distance_previous = []
        distance_index_previous = []
        valid_dist_index = []
        y_center_abs_per_det = []

    # calculate distances and save relevant information



        if y == 0: #beim ersten Bild wird der Abstand der BB zum Folgebild berrechnet

            for x in range(det[y+1]):            
                for i in range(det[y]):
                    distance_next.append(math.sqrt((x_center[sum(det[:y+1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y+1])+x]-y_center[sum(det[:y])+i])**2))
                    distance_index_next.append(i)

            

        elif y > 0 and y < 20: #bei den mittleren Bildern wird der Abstand der BB zum Vorbild und Folgebild berrechnet
            
            for x in range(det[y+1]):
                for i in range(det[y]):
                    distance_next.append(math.sqrt((x_center[sum(det[:y+1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y+1])+x]-y_center[sum(det[:y])+i])**2))
                    distance_index_next.append(i)

            for x in range(det[y-1]):
                for i in range(det[y]):
                    distance_previous.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                    distance_index_previous.append(i)

            
            
        else: # Beim letzten Bild wird der Abstand zur BB des vorletzten Bildes berechnet
            for x in range(det[y-1]):
                
                for i in range(det[y]):
                    distance_previous.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                    distance_index_previous.append(i)


        #print("distance_next: ",distance_next)
        #print("distance_previous:",distance_previous)


    # Movement detector - speichert alle Detections, die der Athlet sein könnte

        if y == 0: # beim ersten Bild nur vergleich mit dem Nächsten und Einschränkung durch eine Vorgabe der Distanz
            #print("first BB")
            for i, x in enumerate(distance_next): # Einschränkung der Detections anhand des Abstandes zum nächsten Bild    
                if 28 < x < 75 and abs(y_center[i%det[y]]-y_center[sum(det[:y+1])+i//det[y]]) < 30:
                    valid_dist_index.append(distance_index_next[i])


        elif athlet_det_index[y-1] == 999: #and len(distance_index_next) != 0: # falls beim letzten Bild keine Detection gemacht wurde 
            #print("last det 999 BP")
            last_det_index = 999
            counter = 0
            for index in athlet_det_index: # letzte gültige detection finden
                if index != 999:
                    last_det_index = index
                    counter = 0
                else:
                    counter = counter + 1

            if last_det_index != 999: # falls bereits valide detection gefunden wurde
                # prüfung zum nächsten Bild
                for i, x in enumerate(distance_next):     
                    if 28 < x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y+1])+i//det[y]]) < 30:
                        valid_dist_index.append(distance_index_next[i])
                # y Abstand zur letzten detection
                for i in range(det[y]):
                    if abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1-counter])+last_det_index]) < 30:
                        valid_dist_index.append(i)
            

            else: # wenn keine vorhanden ist nur eine Einschränkung wie beim ersten Bild
            #eventuell besser nur y Koordinate?
                for i, x in enumerate(distance_next):     
                    if 28 < x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y+1])+i//det[y]]) < 30:
                        valid_dist_index.append(distance_index_next[i])

        elif y > 0 and y < 20: # bei den mittleren Bildern werden beide Richtungen eingeschränkt  und zusätzlich wird auch geprüft, ob die Y Koordinaten nicht zu weit abweichen
            #print("middel BB")
            for i, x in enumerate(distance_next): # Einschränkung der Detections anhand des Abstandes zum nächsten Bild und y Abstand zur letzten detection
                if 28 < x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]) < 30:
                    valid_dist_index.append(distance_index_next[i]) 
      

            for i, x in enumerate(distance_previous): # Einschränkung der Detections anhand des Abstandes zum vorherigen Bild
                if 28 < x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]) < 30:
                    valid_dist_index.append(distance_index_previous[i])
            

        elif y == 20: # beim letzten Bid wieder nur vergleich mit dem vorherigen Bild
            #print("last BB")
            for i, x in enumerate(distance_previous): # Einschränkung der Detections anhand des Abstandes zum vorherigen Bild sowie Validierung der Y Koordinate
                if 28 < x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]) < 30:
                    valid_dist_index.append(distance_index_previous[i])




    # in der Liste valid_dist_index sind die Indexe der validen Detections im Bild gespeichert (Validierung mit Movement detector)
        #print("validdetList: ", valid_dist_index)

    # Anpassungen der Liste der validen Detections - finden des Athleten

        if len(valid_dist_index) > 1: # Falls für eine Bildbox mehrere mögliche Verbindungen zum Folgebild gefunden wurden, werden die Duplikate entfernt
            valid_dist_index = list(OrderedDict.fromkeys(valid_dist_index))

        if len(valid_dist_index) == 1: # wenn nur eine vorhanden ist wird diese verwendet
            athlet_det_index.append(valid_dist_index[0])

        elif len(valid_dist_index) == 0: # falls keine validen aus dem Movement detector vorhanden sind wird 999 (keine detection) ergänzt 
            athlet_det_index.append(999)

        elif y == 0: # Für das erste Bild mit mehreren validen detections
            
            for dist in valid_dist_index: # zentralste detection auf Y Achse
                y_center_abs_per_det.append(abs(y_center[dist]-hight/2))
            closest = min(y_center_abs_per_det)
            min_index = y_center_abs_per_det.index(closest)            
            athlet_det_index.append(valid_dist_index[min_index]) # index der Detection vom ersten Bild gespeichert



        elif y < 21: # Für alle weiteren Bilder mit mehreren validen detections
            dist_to_previous_y_center = [] # Kleinster y Abstand zur letzten detections wird gesucht

            if athlet_det_index[y-1] == 999: # letzes Bild keine detection
                last_det_index = 999
                counter = 0
                for index in athlet_det_index: # letzte gültige detection finden
                    if index != 999:
                        last_det_index = index
                        counter = 0
                    else:
                        counter = counter + 1            
                
                if last_det_index != 999: # falls eine gültige detection gefunden wurde
                    for i, x in enumerate(valid_dist_index): # Abstand der Y Kooridnaten zur letzten Detection
                        dist_to_previous_y_center.append(abs(y_center[sum(det[:y])+x]-y_center[sum(det[:y-1-counter])+last_det_index]))
                    closest = min(dist_to_previous_y_center)
                    min_index = dist_to_previous_y_center.index(closest)
                    athlet_det_index.append(valid_dist_index[min_index]) # index der Detection wird gespeichert

                else: # falls keine gültige detection gefunden wurde - wie bei Bild 1
                    for dist in valid_dist_index:
                        y_center_abs_per_det.append(abs(y_center[dist]-hight/2))                          
                    closest = min(y_center_abs_per_det)
                    min_index = y_center_abs_per_det.index(closest)            
                    athlet_det_index.append(valid_dist_index[min_index]) # index der Detection vom ersten Bild gespeichert

            else:
                for i, x in enumerate(valid_dist_index):
                    dist_to_previous_y_center.append(abs(y_center[sum(det[:y])+x]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]))
                closest = min(dist_to_previous_y_center)
                min_index = dist_to_previous_y_center.index(closest)
                athlet_det_index.append(valid_dist_index[min_index]) # index der Detection wird gespeichert 





      else:
          athlet_det_index.append(999) # falls keine detections im Bild vorhanden sind
      #print("athlet_det_index: ", athlet_det_index)
    print("athlet_det_index: ", athlet_det_index)

    #string = str(athlet_det_index)
    #actualpath = str(Path_Pic)
    #logger.info('Actual Path: %s"', actualpath)
    #logger.info('Ahlet Detection Indexes: "%s" ', string)
      







    #crop images
    for i,x in enumerate(athlet_det_index):
        if x != 999: # zuschneiden laut Koordinaten der BB (50 Pixel vergrößert)
            x_1 = x1[sum(det[:i])+x]-50
            x_2 = x2[sum(det[:i])+x]+50
            y_1 = y1[sum(det[:i])+x]-50
            y_2 = y2[sum(det[:i])+x]+50

            if x_1 < 0:
                x_1 = 0

            if x_2 > width:
                x_2 = width

            if y_1 < 0:
                y_1 = 0

            if y_2 > hight:
                y_2 = hight

        else: # falls keine Detection gefunden wurde wird einfach ein kleines Bild ausgeschnitten, dass später wieder 21 Bilder vorhanden sind
            x_1 = 0
            x_2 = 1
            y_1 = 0
            y_2 = 1


        path = Path_Pic + '/' + str(i+1) + '.jpg'
        print(path)
        im = cv2.imread(path)
        #print(x_1, x_2, y_1, y_2)
        crop = im[int(y_1):int(y_2), int(x_1):int(x_2)] 
        #save_dir.parent.mkdir(parents=True, exist_ok=True)  # make directory
        save_path = Path_Pic + '/exp/' + str(i+1) + '.jpg'
        cv2.imwrite(save_path, crop) # speicher nur den zugeschnittenen Springer



    # hier noch Threshold für mindestens zb 14 Bilder detected einrichten
    #detections = 0
    #for x in athlet_det_index:
        #if x != 999:
            #detections = detections + 1

    #if detections >= 14:

    folder_dir = Path_Pic + '/exp'
    %cd /content/openpose/build/examples/tutorial_api_python
    !python3 04_keypoints_from_images.py --image_dir '{folder_dir}' --keypoint_scale 3 --no_display true

    with open(Path_Pic + '/exp/keypoints.json', "r") as fp:
        keypoints_json = json.load(fp)

    #threshold für 0 Werte in den Keypoints
    #missings = 0
    #for i, value in enumerate(keypoints_json):
        #BP = (i//3)%25
        #if BP not in [15,16,17,18,20,23] and value == 0.0: # unwichtigere Keypoints ausgenommen
            #missings = missings + 1
    #if missings < 100: #nur wenn weniger als 100 Werte fehlen wird die Bilderserie verwendet

    dataset["keypoints"].append(keypoints_json)

    if l == 'NoTelemark':
        dataset["labels"].append(0)
    if l == 'Telemark':
        dataset["labels"].append(1) 


with open(json_path, "w") as fp:
  json.dump(dataset, fp, indent=4)


Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
/content.gdrive/My Drive/Manually_Labeled_09042022/NoTelemark/647/13.jpg
/content.gdrive/My Drive/Manually_Labeled_09042022/NoTelemark/647/14.jpg
/content.gdrive/My Drive/Manually_Labeled_09042022/NoTelemark/647/15.jpg
/content.gdrive/My Drive/Manually_Labeled_09042022/NoTelemark/647/16.jpg
/content.gdrive/My Drive/Manually_Labeled_09042022/NoTelemark/647/17.jpg
/content.gdrive/My Drive/Manually_Labeled_09042022/NoTelemark/647/18.jpg
/content.gdrive/My Drive/Manually_Labeled_09042022/NoTelemark/647/19.jpg
/content.gdrive/My Drive/Manually_Labeled_09042022/NoTelemark/647/20.jpg
/content.gdrive/My Drive/Manually_Labeled_09042022/NoTelemark/647/21.jpg
/content/openpose/build/examples/tutorial_api_python
Starting OpenPose Python Wrapper...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
['/content.gdrive/My Drive/Manually_Labeled_09042022/NoTelemark/647/exp/1.jpg', '/content.gdrive

In [ ]:
#try 07.04.2022 17:52 (mark12)

#Update
#Threshold für die Bilder

#sollte nun final sein - Test 02.04.2022 16:53 - erfolgreich

import os
from os.path import exists, join, basename, splitext
import math
from collections import OrderedDict
import json
import numpy as np
import cv2


#Aufbau Json File genau anschauen

json_path = "/content.gdrive/My Drive/Test/data.json"

# dictionary to store mapping, labels, and Keypoints
dataset = {
    "labels":[],
    "keypoints":[]
}

PATH = "/content.gdrive/My Drive/Test"
label= []
try:
    label = next(os.walk(PATH))[1]
except StopIteration:
    pass
#data["labels"].append(label)
print(label)

for l in label:
  Path_jumpers = PATH + '/' + l
  print(Path_jumpers)
  Jumpers = []
  try:
    Jumpers = next(os.walk(Path_jumpers))[1]
  except StopIteration:
    pass
  print(Jumpers)

  for j in Jumpers:
    Path_Pic = Path_jumpers + '/' + j
    print(Path_Pic)

    %cd /content/yolov5
    !python detect.py --save-crop --source '{Path_Pic}' --project '{Path_Pic}' --save-txt


    with open(Path_Pic + '/exp/predictions.json', "r") as fp:
        data = json.load(fp)

    # convert lists to numpy arrays
    x_center = np.array(data["x_center"])
    y_center = np.array(data["y_center"])
    x1 = np.array(data["x1"])
    x2 = np.array(data["x2"])
    y1 = np.array(data["y1"])
    y2 = np.array(data["y2"])
    det = np.array(data["det"])
    width = 1920
    hight = 1080




    #detect movement mit check vor und zurück


    athlet_det_index = []


    for y in range(21):
      print("loop" , y)
      if det[y] != 0: #wird eventuellenötigt, falls keine detections gemacht wurden
        print("detvorhanden")
        distance_next = []
        distance_index_next = []
        distance_previous = []
        distance_index_previous = []
        valid_dist_index = []
        y_center_abs_per_det = []

    # calculate distances and save relevant information



        if y == 0: #beim ersten Bild wird der Abstand der BB zum Folgebild berrechnet

            for x in range(det[y+1]):            
                for i in range(det[y]):
                    distance_next.append(math.sqrt((x_center[sum(det[:y+1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y+1])+x]-y_center[sum(det[:y])+i])**2))
                    distance_index_next.append(i)

            

        elif y > 0 and y < 20: #bei den mittleren Bildern wird der Abstand der BB zum Vorbild und Folgebild berrechnet
            
            for x in range(det[y+1]):
                for i in range(det[y]):
                    distance_next.append(math.sqrt((x_center[sum(det[:y+1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y+1])+x]-y_center[sum(det[:y])+i])**2))
                    distance_index_next.append(i)

            for x in range(det[y-1]):
                for i in range(det[y]):
                    distance_previous.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                    distance_index_previous.append(i)

            
            
        else: # Beim letzten Bild wird der Abstand zur BB des vorletzten Bildes berechnet
            for x in range(det[y-1]):
                
                for i in range(det[y]):
                    distance_previous.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                    distance_index_previous.append(i)


        print("distance_next: ",distance_next)
        print("distance_previous:",distance_previous)


    # Movement detector - speichert alle Detections, die der Athlet sein könnte

        if y == 0: # beim ersten Bild nur vergleich mit dem Nächsten und Einschränkung durch eine Vorgabe der Distanz
            print("first BB")
            for i, x in enumerate(distance_next): # Einschränkung der Detections anhand des Abstandes zum nächsten Bild    
                if 28 < x < 75 and abs(y_center[i%det[y]]-y_center[sum(det[:y+1])+i//det[y]]) < 30:
                    valid_dist_index.append(distance_index_next[i])


        elif athlet_det_index[y-1] == 999: #and len(distance_index_next) != 0: # falls beim letzten Bild keine Detection gemacht wurde 
            print("last det 999 BP")
            last_det_index = 999
            counter = 0
            for index in athlet_det_index: # letzte gültige detection finden
                if index != 999:
                    last_det_index = index
                    counter = 0
                else:
                    counter = counter + 1

            if last_det_index != 999: # falls bereits valide detection gefunden wurde
                # prüfung zum nächsten Bild
                for i, x in enumerate(distance_next):     
                    if 28 < x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y+1])+i//det[y]]) < 30:
                        valid_dist_index.append(distance_index_next[i])
                # y Abstand zur letzten detection
                for i in range(det[y]):
                    if abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1-counter])+last_det_index]) < 30:
                        valid_dist_index.append(i)
            

            else: # wenn keine vorhanden ist nur eine Einschränkung wie beim ersten Bild
            #eventuell besser nur y Koordinate?
                for i, x in enumerate(distance_next):     
                    if 28 < x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y+1])+i//det[y]]) < 30:
                        valid_dist_index.append(distance_index_next[i])

        elif y > 0 and y < 20: # bei den mittleren Bildern werden beide Richtungen eingeschränkt  und zusätzlich wird auch geprüft, ob die Y Koordinaten nicht zu weit abweichen
            print("middel BB")
            for i, x in enumerate(distance_next): # Einschränkung der Detections anhand des Abstandes zum nächsten Bild und y Abstand zur letzten detection
                if 28 < x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]) < 30:
                    valid_dist_index.append(distance_index_next[i]) 
      

            for i, x in enumerate(distance_previous): # Einschränkung der Detections anhand des Abstandes zum vorherigen Bild
                if 28 < x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]) < 30:
                    valid_dist_index.append(distance_index_previous[i])
            

        elif y == 20: # beim letzten Bid wieder nur vergleich mit dem vorherigen Bild
            print("last BB")
            for i, x in enumerate(distance_previous): # Einschränkung der Detections anhand des Abstandes zum vorherigen Bild sowie Validierung der Y Koordinate
                if 28 < x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]) < 30:
                    valid_dist_index.append(distance_index_previous[i])




    # in der Liste valid_dist_index sind die Indexe der validen Detections im Bild gespeichert (Validierung mit Movement detector)
        print("validdetList: ", valid_dist_index)

    # Anpassungen der Liste der validen Detections - finden des Athleten

        if len(valid_dist_index) > 1: # Falls für eine Bildbox mehrere mögliche Verbindungen zum Folgebild gefunden wurden, werden die Duplikate entfernt
            valid_dist_index = list(OrderedDict.fromkeys(valid_dist_index))

        if len(valid_dist_index) == 1: # wenn nur eine vorhanden ist wird diese verwendet
            athlet_det_index.append(valid_dist_index[0])

        elif len(valid_dist_index) == 0: # falls keine validen aus dem Movement detector vorhanden sind wird 999 (keine detection) ergänzt 
            athlet_det_index.append(999)

        elif y == 0: # Für das erste Bild mit mehreren validen detections
            
            for dist in valid_dist_index: # zentralste detection auf Y Achse
                y_center_abs_per_det.append(abs(y_center[dist]-hight/2))
            closest = min(y_center_abs_per_det)
            min_index = y_center_abs_per_det.index(closest)            
            athlet_det_index.append(valid_dist_index[min_index]) # index der Detection vom ersten Bild gespeichert



        elif y < 21: # Für alle weiteren Bilder mit mehreren validen detections
            dist_to_previous_y_center = [] # Kleinster y Abstand zur letzten detections wird gesucht

            if athlet_det_index[y-1] == 999: # letzes Bild keine detection
                last_det_index = 999
                counter = 0
                for index in athlet_det_index: # letzte gültige detection finden
                    if index != 999:
                        last_det_index = index
                        counter = 0
                    else:
                        counter = counter + 1            
                
                if last_det_index != 999: # falls eine gültige detection gefunden wurde
                    for i, x in enumerate(valid_dist_index): # Abstand der Y Kooridnaten zur letzten Detection
                        dist_to_previous_y_center.append(abs(y_center[sum(det[:y])+x]-y_center[sum(det[:y-1-counter])+last_det_index]))
                    closest = min(dist_to_previous_y_center)
                    min_index = dist_to_previous_y_center.index(closest)
                    athlet_det_index.append(valid_dist_index[min_index]) # index der Detection wird gespeichert

                else: # falls keine gültige detection gefunden wurde - wie bei Bild 1
                    for dist in valid_dist_index:
                        y_center_abs_per_det.append(abs(y_center[dist]-hight/2))                          
                    closest = min(y_center_abs_per_det)
                    min_index = y_center_abs_per_det.index(closest)            
                    athlet_det_index.append(valid_dist_index[min_index]) # index der Detection vom ersten Bild gespeichert

            else:
                for i, x in enumerate(valid_dist_index):
                    dist_to_previous_y_center.append(abs(y_center[sum(det[:y])+x]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]))
                closest = min(dist_to_previous_y_center)
                min_index = dist_to_previous_y_center.index(closest)
                athlet_det_index.append(valid_dist_index[min_index]) # index der Detection wird gespeichert 





      else:
          athlet_det_index.append(999) # falls keine detections im Bild vorhanden sind
      print("athlet_det_index: ", athlet_det_index)
    print("athlet_det_index: ", athlet_det_index)

    #string = str(athlet_det_index)
    #actualpath = str(Path_Pic)
    #logger.info('Actual Path: %s"', actualpath)
    #logger.info('Ahlet Detection Indexes: "%s" ', string)
      







    #crop images
    for i,x in enumerate(athlet_det_index):
        if x != 999: # zuschneiden laut Koordinaten der BB (50 Pixel vergrößert)
            x_1 = x1[sum(det[:i])+x]-50
            x_2 = x2[sum(det[:i])+x]+50
            y_1 = y1[sum(det[:i])+x]-50
            y_2 = y2[sum(det[:i])+x]+50

            if x_1 < 0:
                x_1 = 0

            if x_2 > width:
                x_2 = width

            if y_1 < 0:
                y_1 = 0

            if y_2 > hight:
                y_2 = hight

        else: # falls keine Detection gefunden wurde wird einfach ein kleines Bild ausgeschnitten, dass später wieder 21 Bilder vorhanden sind
            x_1 = 0
            x_2 = 1
            y_1 = 0
            y_2 = 1


        path = Path_Pic + '/' + str(i+1) + '.jpg'
        print(path)
        im = cv2.imread(path)
        #print(x_1, x_2, y_1, y_2)
        crop = im[int(y_1):int(y_2), int(x_1):int(x_2)] 
        #save_dir.parent.mkdir(parents=True, exist_ok=True)  # make directory
        save_path = Path_Pic + '/exp/' + str(i+1) + '.jpg'
        cv2.imwrite(save_path, crop) # speicher nur den zugeschnittenen Springer



    # hier noch Threshold für mindestens zb 14 Bilder detected einrichten
    detections = 0
    for x in athlet_det_index:
        if x != 999:
            detections = detections + 1

    if detections >= 14:
        print("!!VALIDE DET")
        folder_dir = Path_Pic + '/exp'
        %cd /content/openpose/build/examples/tutorial_api_python
        !python3 04_keypoints_from_images.py --image_dir '{folder_dir}' --keypoint_scale 3 --no_display true

        with open(Path_Pic + '/exp/keypoints.json', "r") as fp:
            keypoints_json = json.load(fp)

        #threshold für 0 Werte in den Keypoints
        missings = 0
        for i, value in enumerate(keypoints_json):
            BP = (i//3)%25
            if BP not in [15,16,17,18,20,23] and value == 0.0: # unwichtigere Keypoints ausgenommen
                missings = missings + 1
        if missings < 100: #nur wenn weniger als 100 Werte fehlen wird die Bilderserie verwendet
            print("!!VALID KEYPOINTS")
            dataset["keypoints"].append(keypoints_json)

            if l == 'NoTelemark':
                dataset["labels"].append(0)
            if l == 'Telemark':
                dataset["labels"].append(1) 


with open(json_path, "w") as fp:
  json.dump(dataset, fp, indent=4)


In [3]:
#try 02.04.2022 10:52 (mark12)

#Update
# 28 < x < 75 
# abs < 30
# Lückenhafte Bilder werden besser

#Mirror und fehlerhafte

import os
from os.path import exists, join, basename, splitext
import math
from collections import OrderedDict
import json
import numpy as np
import cv2


#Aufbau Json File genau anschauen

json_path = "/content.gdrive/My Drive/Manually_Labeled_09042022/data_Manually_Labeled_mirror_09042022.json"

# dictionary to store mapping, labels, and Keypoints
dataset = {
    "labels":[],
    "keypoints":[]
}

PATH = "/content.gdrive/My Drive/Manually_Labeled_09042022"
label= []
try:
    label = next(os.walk(PATH))[1]
except StopIteration:
    pass
#data["labels"].append(label)
print(label)

for l in label:
  Path_jumpers = PATH + '/' + l
  print(Path_jumpers)
  Jumpers = []
  try:
    Jumpers = next(os.walk(Path_jumpers))[1]
  except StopIteration:
    pass
  print("Anzahl: " , len(Jumpers))

  for j in Jumpers:
    Path_Pic = Path_jumpers + '/' + j
    print(Path_Pic)

    Path_check = Path_Pic + '/exp'


    if os.path.isdir(Path_check):

        print("Done")
    else:

        %cd /content/yolov5
        !python detect.py --save-crop --source '{Path_Pic}' --project '{Path_Pic}' --save-txt


        with open(Path_Pic + '/exp/predictions.json', "r") as fp:
            data = json.load(fp)

        # convert lists to numpy arrays
        x_center = np.array(data["x_center"])
        y_center = np.array(data["y_center"])
        x1 = np.array(data["x1"])
        x2 = np.array(data["x2"])
        y1 = np.array(data["y1"])
        y2 = np.array(data["y2"])
        det = np.array(data["det"])
        width = 1920
        hight = 1080




        #detect movement mit check vor und zurück


        athlet_det_index = []


        for y in range(21):
          #print("loop" , y)
          if det[y] != 0: #wird eventuellenötigt, falls keine detections gemacht wurden
            #print("detvorhanden")
            distance_next = []
            distance_index_next = []
            distance_previous = []
            distance_index_previous = []
            valid_dist_index = []
            y_center_abs_per_det = []

        # calculate distances and save relevant information



            if y == 0: #beim ersten Bild wird der Abstand der BB zum Folgebild berrechnet

                for x in range(det[y+1]):            
                    for i in range(det[y]):
                        distance_next.append(math.sqrt((x_center[sum(det[:y+1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y+1])+x]-y_center[sum(det[:y])+i])**2))
                        distance_index_next.append(i)

                

            elif y > 0 and y < 20: #bei den mittleren Bildern wird der Abstand der BB zum Vorbild und Folgebild berrechnet
                
                for x in range(det[y+1]):
                    for i in range(det[y]):
                        distance_next.append(math.sqrt((x_center[sum(det[:y+1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y+1])+x]-y_center[sum(det[:y])+i])**2))
                        distance_index_next.append(i)

                for x in range(det[y-1]):
                    for i in range(det[y]):
                        distance_previous.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                        distance_index_previous.append(i)

                
                
            else: # Beim letzten Bild wird der Abstand zur BB des vorletzten Bildes berechnet
                for x in range(det[y-1]):
                    
                    for i in range(det[y]):
                        distance_previous.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                        distance_index_previous.append(i)


            #print("distance_next: ",distance_next)
            #print("distance_previous:",distance_previous)


        # Movement detector - speichert alle Detections, die der Athlet sein könnte

            if y == 0: # beim ersten Bild nur vergleich mit dem Nächsten und Einschränkung durch eine Vorgabe der Distanz
                #print("first BB")
                for i, x in enumerate(distance_next): # Einschränkung der Detections anhand des Abstandes zum nächsten Bild    
                    if 28 < x < 75 and abs(y_center[i%det[y]]-y_center[sum(det[:y+1])+i//det[y]]) < 30:
                        valid_dist_index.append(distance_index_next[i])


            elif athlet_det_index[y-1] == 999: #and len(distance_index_next) != 0: # falls beim letzten Bild keine Detection gemacht wurde 
                #print("last det 999 BP")
                last_det_index = 999
                counter = 0
                for index in athlet_det_index: # letzte gültige detection finden
                    if index != 999:
                        last_det_index = index
                        counter = 0
                    else:
                        counter = counter + 1

                if last_det_index != 999: # falls bereits valide detection gefunden wurde
                    # prüfung zum nächsten Bild
                    for i, x in enumerate(distance_next):     
                        if 28 < x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y+1])+i//det[y]]) < 30:
                            valid_dist_index.append(distance_index_next[i])
                    # y Abstand zur letzten detection
                    for i in range(det[y]):
                        if abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1-counter])+last_det_index]) < 30:
                            valid_dist_index.append(i)
                

                else: # wenn keine vorhanden ist nur eine Einschränkung wie beim ersten Bild
                #eventuell besser nur y Koordinate?
                    for i, x in enumerate(distance_next):     
                        if 28 < x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y+1])+i//det[y]]) < 30:
                            valid_dist_index.append(distance_index_next[i])

            elif y > 0 and y < 20: # bei den mittleren Bildern werden beide Richtungen eingeschränkt  und zusätzlich wird auch geprüft, ob die Y Koordinaten nicht zu weit abweichen
                #print("middel BB")
                for i, x in enumerate(distance_next): # Einschränkung der Detections anhand des Abstandes zum nächsten Bild und y Abstand zur letzten detection
                    if 28 < x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]) < 30:
                        valid_dist_index.append(distance_index_next[i]) 
          

                for i, x in enumerate(distance_previous): # Einschränkung der Detections anhand des Abstandes zum vorherigen Bild
                    if 28 < x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]) < 30:
                        valid_dist_index.append(distance_index_previous[i])
                

            elif y == 20: # beim letzten Bid wieder nur vergleich mit dem vorherigen Bild
                #print("last BB")
                for i, x in enumerate(distance_previous): # Einschränkung der Detections anhand des Abstandes zum vorherigen Bild sowie Validierung der Y Koordinate
                    if 28 < x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]) < 30:
                        valid_dist_index.append(distance_index_previous[i])




        # in der Liste valid_dist_index sind die Indexe der validen Detections im Bild gespeichert (Validierung mit Movement detector)
            #print("validdetList: ", valid_dist_index)

        # Anpassungen der Liste der validen Detections - finden des Athleten

            if len(valid_dist_index) > 1: # Falls für eine Bildbox mehrere mögliche Verbindungen zum Folgebild gefunden wurden, werden die Duplikate entfernt
                valid_dist_index = list(OrderedDict.fromkeys(valid_dist_index))

            if len(valid_dist_index) == 1: # wenn nur eine vorhanden ist wird diese verwendet
                athlet_det_index.append(valid_dist_index[0])

            elif len(valid_dist_index) == 0: # falls keine validen aus dem Movement detector vorhanden sind wird 999 (keine detection) ergänzt 
                athlet_det_index.append(999)

            elif y == 0: # Für das erste Bild mit mehreren validen detections
                
                for dist in valid_dist_index: # zentralste detection auf Y Achse
                    y_center_abs_per_det.append(abs(y_center[dist]-hight/2))
                closest = min(y_center_abs_per_det)
                min_index = y_center_abs_per_det.index(closest)            
                athlet_det_index.append(valid_dist_index[min_index]) # index der Detection vom ersten Bild gespeichert



            elif y < 21: # Für alle weiteren Bilder mit mehreren validen detections
                dist_to_previous_y_center = [] # Kleinster y Abstand zur letzten detections wird gesucht

                if athlet_det_index[y-1] == 999: # letzes Bild keine detection
                    last_det_index = 999
                    counter = 0
                    for index in athlet_det_index: # letzte gültige detection finden
                        if index != 999:
                            last_det_index = index
                            counter = 0
                        else:
                            counter = counter + 1            
                    
                    if last_det_index != 999: # falls eine gültige detection gefunden wurde
                        for i, x in enumerate(valid_dist_index): # Abstand der Y Kooridnaten zur letzten Detection
                            dist_to_previous_y_center.append(abs(y_center[sum(det[:y])+x]-y_center[sum(det[:y-1-counter])+last_det_index]))
                        closest = min(dist_to_previous_y_center)
                        min_index = dist_to_previous_y_center.index(closest)
                        athlet_det_index.append(valid_dist_index[min_index]) # index der Detection wird gespeichert

                    else: # falls keine gültige detection gefunden wurde - wie bei Bild 1
                        for dist in valid_dist_index:
                            y_center_abs_per_det.append(abs(y_center[dist]-hight/2))                          
                        closest = min(y_center_abs_per_det)
                        min_index = y_center_abs_per_det.index(closest)            
                        athlet_det_index.append(valid_dist_index[min_index]) # index der Detection vom ersten Bild gespeichert

                else:
                    for i, x in enumerate(valid_dist_index):
                        dist_to_previous_y_center.append(abs(y_center[sum(det[:y])+x]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]))
                    closest = min(dist_to_previous_y_center)
                    min_index = dist_to_previous_y_center.index(closest)
                    athlet_det_index.append(valid_dist_index[min_index]) # index der Detection wird gespeichert 





          else:
              athlet_det_index.append(999) # falls keine detections im Bild vorhanden sind
          #print("athlet_det_index: ", athlet_det_index)
        print("athlet_det_index: ", athlet_det_index)

        #string = str(athlet_det_index)
        #actualpath = str(Path_Pic)
        #logger.info('Actual Path: %s"', actualpath)
        #logger.info('Ahlet Detection Indexes: "%s" ', string)
          







        #crop images
        for i,x in enumerate(athlet_det_index):
            if x != 999: # zuschneiden laut Koordinaten der BB (50 Pixel vergrößert)
                x_1 = x1[sum(det[:i])+x]-50
                x_2 = x2[sum(det[:i])+x]+50
                y_1 = y1[sum(det[:i])+x]-50
                y_2 = y2[sum(det[:i])+x]+50

                if x_1 < 0:
                    x_1 = 0

                if x_2 > width:
                    x_2 = width

                if y_1 < 0:
                    y_1 = 0

                if y_2 > hight:
                    y_2 = hight

            else: # falls keine Detection gefunden wurde wird einfach ein kleines Bild ausgeschnitten, dass später wieder 21 Bilder vorhanden sind
                x_1 = 0
                x_2 = 1
                y_1 = 0
                y_2 = 1


            path = Path_Pic + '/' + str(i+1) + '.jpg'
            #print(path)
            im = cv2.imread(path)
            #print(x_1, x_2, y_1, y_2)
            crop = im[int(y_1):int(y_2), int(x_1):int(x_2)] 
            #save_dir.parent.mkdir(parents=True, exist_ok=True)  # make directory
            save_path = Path_Pic + '/exp/' + str(i+1) + '.jpg'
            cv2.imwrite(save_path, crop) # speicher nur den zugeschnittenen Springer



        # hier noch Threshold für mindestens zb 14 Bilder detected einrichten
        #detections = 0
        #for x in athlet_det_index:
            #if x != 999:
                #detections = detections + 1

        #if detections >= 14:

        folder_dir = Path_Pic + '/exp'
        %cd /content/openpose/build/examples/tutorial_api_python
        !python3 04_keypoints_from_images.py --image_dir '{folder_dir}' --keypoint_scale 3 --no_display true

        with open(Path_Pic + '/exp/keypoints.json', "r") as fp:
            keypoints_json = json.load(fp)

        #threshold für 0 Werte in den Keypoints
        #missings = 0
        #for i, value in enumerate(keypoints_json):
            #BP = (i//3)%25
            #if BP not in [15,16,17,18,20,23] and value == 0.0: # unwichtigere Keypoints ausgenommen
                #missings = missings + 1
        #if missings < 100: #nur wenn weniger als 100 Werte fehlen wird die Bilderserie verwendet

        dataset["keypoints"].append(keypoints_json)

        if l == 'NoTelemark':
            dataset["labels"].append(0)
        if l == 'Telemark':
            dataset["labels"].append(1) 


with open(json_path, "w") as fp:
  json.dump(dataset, fp, indent=4)


['Telemark', 'NoTelemark']
/content.gdrive/My Drive/Manually_Labeled_09042022/Telemark
Anzahl:  122
/content.gdrive/My Drive/Manually_Labeled_09042022/Telemark/7711
/content/yolov5
YOLOv5 🚀 v6.1-35-g6c6f664 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

100% 14.1M/14.1M [00:00<00:00, 107MB/s] 

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients, 16.5 GFLOPs
BP  /content.gdrive/My Drive/Manually_Labeled_09042022/Telemark/7711
images:  ['/content.gdrive/My Drive/Manually_Labeled_09042022/Telemark/7711/1.jpg', '/content.gdrive/My Drive/Manually_Labeled_09042022/Telemark/7711/2.jpg', '/content.gdrive/My Drive/Manually_Labeled_09042022/Telemark/7711/3.jpg', '/content.gdrive/My Drive/Manually_Labeled_09042022/Telemark/7711/4.jpg', '/content.gdrive/My Drive/Manually_Labeled_09042022/Telemark/7711/5.jpg', '/content.gdrive/My Drive/Manually_Labeled_09042022/Telemark/7711/6.jpg', '/content.gdrive/My Drive/Manually_Labeled_09042022/Telemark/7711/7.jpg', '/content.gdriv

    Path_check = Path_Pic + '/exp'


    if os.path.isdir(Path_check):

        print("Done")
    else:

In [4]:
#test load data
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

with open("/content.gdrive/My Drive/Manually_Labeled_09042022/data_Manually_Labeled_mirror_09042022.json", "r") as fp:
    data = json.load(fp)

# convert lists to numpy arrays
X = np.array(data["keypoints"])
#X = np.array(data["mfcc"])
y = np.array(data["labels"])

print(X)
print(X.shape)
print(y)
print(y.shape)

for i in range(4):
    count = 0 
    for y in range(1575):
        if X[i][y] == 0:
            count = count +1
    print(count)
            
print(X[0][0])
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

[[0.29911649 0.27518421 0.29718834 ... 0.44530392 0.6671012  0.43639362]
 [0.67712164 0.34077457 0.30543998 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.56176138 0.75441986 0.60075879]
 ...
 [0.3605116  0.29570448 0.17717955 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.6170072  0.64117408 0.27421421]
 [0.40025148 0.31983036 0.38911289 ... 0.4639746  0.70519263 0.15024978]]
(264, 1575)
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0]
(264,)
